
## Imports

The above imports are not at all mandatory in order to have the following code work. This cell is here to ensure that your setup is correct. 

In [ ]:
%cd ..

%load_ext autoreload
%autoreload 2

## DDPG Agent on pendulum 


In [ ]:
import acme

from src.environments.inverted_pendulum import InvertedPendulumEnv, inverted_pendulum_env_factory
from src.agents import DDPGAgent
from src.interaction_loops import ddpg_parallel_interaction_loop, evaluation_parallel_interaction_loop

In [ ]:
pendulum_environment = InvertedPendulumEnv(for_evaluation=True)  #### WARNING, I changed the env
pendulum_environment_spec = acme.make_environment_spec(pendulum_environment)

In [31]:
# define the agent
num_evaluation_steps = 1000
max_learner_steps=50
pendulum_agent = DDPGAgent(seed=0, 
learning_rate=5e-4,
gamma=.95, 
environment_spec=pendulum_environment_spec)

(1, 1)


In [32]:
ddpg_parallel_interaction_loop(pendulum_agent, env_factory=inverted_pendulum_env_factory, max_learner_steps=max_learner_steps, num_actors=16, batch_size=32, buffer_size=1000)


TypeError: state argument does not appear valid. It should be a mapping but is of type <class 'src.data.Transition'>. For reference the parameters for apply are `apply(params, rng, ...)`` for `hk.transform` and `apply(params, state, rng, ...)` for `hk.transform_with_state`.
The argument was: Transition(obs_tm1=Traced<ShapedArray(float32[15,3])>with<DynamicJaxprTrace(level=0/1)>, action_tm1=Traced<ShapedArray(float32[15,1])>with<DynamicJaxprTrace(level=0/1)>, reward_t=Traced<ShapedArray(float32[15])>with<DynamicJaxprTrace(level=0/1)>, discount_t=Traced<ShapedArray(float32[15])>with<DynamicJaxprTrace(level=0/1)>, obs_t=Traced<ShapedArray(float32[15,3])>with<DynamicJaxprTrace(level=0/1)>, done=Traced<ShapedArray(int32[15])>with<DynamicJaxprTrace(level=0/1)>).

In [ ]:
jnp.expand_dims()

In [ ]:
for _ in range(10):
  ddpg_parallel_interaction_loop(pendulum_agent, env_factory=inverted_pendulum_env_factory, max_learner_steps=max_learner_steps, sequence_length=sequence_length, num_actors=16)
  plot_policy_on_pendulum(pendulum_agent, 50)
  evaluation_parallel_interaction_loop(pendulum_agent, env_factory=inverted_pendulum_env_factory, sequence_length=200, num_actors=4)

In [ ]:
from src.networks import ValueNetworkDDPG, PolicyNetworkDDPG, PolicyNetwork
from src.data import Transition
import jax.numpy as jnp
import jax
import haiku as hk

In [26]:
from src.data import Transition
observation = pendulum_environment_spec.observations.generate_value()
action = pendulum_environment_spec.actions.generate_value()

tr = Transition(
    obs_tm1= observation[None],
    action_tm1= action[None],
    reward_t= jnp.zeros(1)[None],
    discount_t=jnp.zeros(1)[None],
    obs_t=observation[None],
    done=jnp.zeros(1)[None])

In [ ]:
jnp.concatenate([tr.obs_t, tr.action_tm1], axis=1)

In [ ]:
def f(transition):
    action = PolicyNetworkDDPG(environment_spec.actions)(transition.obs_tm1, True)
    return action

def f1(obs):
    return hk.BatchApply(PolicyNetworkDDPG(environment_spec.actions))(obs, True)
    
init_loss, apply_loss = hk.without_apply_rng(hk.transform_with_state(f))

In [ ]:
rng = jax.random.PRNGKey(seed=0)

In [ ]:
init_loss(rng,tr)

In [ ]:
tr.obs_tm1